In [121]:
#Stop loss =price-ATR ATR will Be large enought when your Market very volatile and small ATR for less volatility
#Take profit =price+Take profit(TP)stop loss ration(SL) Ratio*ATR Take profit is towice as the stop loss
#TPSLRation=2Ecch win makes up for 2 losses
#If it Touch Stop loss firs it is a winning Trade and touch Take profit first it is a winning Trade
#!python -m pip install pandas_ta

In [122]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import yfinance as yf

In [123]:
df= yf.Ticker('FB').history(start='2020-01-01',end='2022-08-07')
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-12-31,203.994995,205.559998,203.600098,205.250000,8962756,0,0
2020-01-02,206.750000,209.789993,206.270004,209.779999,12090707,0,0
2020-01-03,207.213501,210.399994,206.949997,208.669998,11197416,0,0
2020-01-06,206.699997,212.779999,206.520004,212.600006,17073861,0,0
2020-01-07,212.820007,214.580002,211.750000,213.059998,14604985,0,0
...,...,...,...,...,...,...,...
2022-07-11,167.065002,167.490005,161.910004,162.880005,21905133,0,0
2022-07-12,164.800003,165.910706,162.100006,163.270004,16639659,0,0
2022-07-13,160.160004,164.979996,159.612701,163.490005,16555138,0,0


In [124]:
df['ATR']=df.ta.atr(length=10)
df['RSI']=df.ta.rsi()#length is by default 14 days
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ATR,RSI
Date,,,,,,,,,
2020-01-22,222.309998,222.750000,221.279999,221.320007,12212814,0,0,2.915702,77.259204
2020-01-23,220.750000,221.485001,219.270004,219.759995,11195000,0,0,2.827464,69.321583
2020-01-24,220.800003,221.779999,216.110001,217.940002,11905824,0,0,3.176370,61.396403
2020-01-27,213.100006,216.059998,212.500000,214.869995,13810655,0,0,3.448040,50.838293
2020-01-28,216.139999,218.740005,213.520004,217.789993,13413835,0,0,3.656530,58.200993
...,...,...,...,...,...,...,...,...,...
2022-07-11,167.065002,167.490005,161.910004,162.880005,21905133,0,0,7.595914,42.568779
2022-07-12,164.800003,165.910706,162.100006,163.270004,16639659,0,0,7.217393,42.925283
2022-07-13,160.160004,164.979996,159.612701,163.490005,16555138,0,0,7.032383,43.139706


In [125]:
#for down trends shooting star
#for up trendes hammer
def Revsignal1(df1):
    
    length = len(df1)
    high = list(df1['High'])
    low = list(df1['Low'])
    close = list(df1['Close'])
    open = list(df1['Open'])
    signal = [0] * length
    highdiff = [0] * length
    lowdiff = [0] * length
    bodydiff = [0] * length
    ratio1 = [0] * length
    ratio2 = [0] * length
        
    for row in range(0, length):
        
        highdiff[row] = high[row]-max(open[row],close[row])
        bodydiff[row] = abs(open[row]-close[row])
        if bodydiff[row]<0.002:
            bodydiff[row]=0.002
        lowdiff[row] = min(open[row],close[row])-low[row]
        ratio1[row] = highdiff[row]/bodydiff[row]
        ratio2[row] = lowdiff[row]/bodydiff[row]
        #print(df.RSI[row])
        #  |
        # _|_
        #|__|
        # |
        # 
        #finding our shooting stars
        if (ratio1[row]>2.5 and lowdiff[row]<0.3*highdiff[row] and bodydiff[row]>0.03 and df.RSI[row]>50 and df.RSI[row]<70 ):
            signal[row] = 1
        
        
        # _|_
        #|__|
        # |
        # |
        # finding our hammer
        elif (ratio2[row]>2.5 and highdiff[row]<0.23*lowdiff[row] and bodydiff[row]>0.03 and df.RSI[row]<55 and df.RSI[row]>30):
            signal[row]=2
    return signal
df['signal1'] = Revsignal1(df)
df[df['signal1']==1].count()#selling signals

Open            18
High            18
Low             18
Close           18
Volume          18
Dividends       18
Stock Splits    18
ATR             18
RSI             18
signal1         18
dtype: int64

In [126]:
#Test our strategy based on uptrend and downdtrend signals
def mytarget(barsupfront,df):#barsupfrons determines how many period i will check after hammer or shooting stars to check ,it will take profit fist or stoploss 
    length = len(df)
    high = list(df['High'])
    low = list(df['Low'])
    close = list(df['Close'])
    opens = list(df['Open'])
    datr = list(df['ATR'])
    trendcat = [0] * length
     
    for line in range(0,length-barsupfront-1):#this loop will continue untill last 100 period
        valueOpenLow=0
        valueOpenHigh=0
        
        highdiff = high[line]-max(opens[line],close[line])
        bodydiff = abs(opens[line]-close[line])
        
        pipdiff = datr[line]*1. #maintaining minimum distance for my stop loss i dont want it too close
        if pipdiff<1.1:
            pipdiff=1.1
            
        SLTPRatio = 2. #pipdiff*Ratio gives Tp
        for i in range(1,barsupfront+1):
            value1 = close[line]-low[line+i]
            value2 = close[line]-high[line+i]
            valueOpenLow = max(value1, valueOpenLow)
            valueOpenHigh = min(value2, valueOpenHigh)

            if ( (valueOpenLow >= (SLTPRatio*pipdiff) ) and (-valueOpenHigh < pipdiff) ):
                trendcat[line] = 1 #-1 downtrend touched take profit without touthing stop loss
                break
            elif ((valueOpenLow < pipdiff) ) and (-valueOpenHigh >= (SLTPRatio*pipdiff)):
                trendcat[line] = 2 # uptrend
                break 
            else:
                trendcat[line] = 0 # no clear trend
            
    return trendcat

# mytarget(barsfront to take into account, dataframe)
df['Trend'] = mytarget(100, df)
df.tail(20)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ATR,RSI,signal1,Trend
Date,,,,,,,,,,,
2022-06-16,163.720001,165.080002,159.610001,160.869995,26944143,0,0,9.476935,33.900174,0,0
2022-06-17,161.679993,165.899994,159.899994,163.740005,31374215,0,0,9.129242,36.208781,0,0
2022-06-21,165.929993,167.759995,155.970001,157.050003,50724899,0,0,9.395317,33.290065,0,0
2022-06-22,158.380005,160.789993,155.440002,155.850006,47596820,0,0,8.990784,32.779653,0,0
2022-06-23,156.270004,159.750000,154.250000,158.750000,40499198,0,0,8.641706,35.359035,0,0
2022-06-24,161.725006,170.250000,161.300003,170.160004,68736009,0,0,8.927535,44.399075,0,0
2022-06-27,171.320007,171.750000,168.009995,169.490005,29174559,0,0,8.408782,44.009864,0,0
2022-06-28,169.899994,171.300003,160.610001,160.679993,27788725,0,0,8.636904,39.149936,0,0
2022-06-29,163.570007,166.330002,160.320007,163.940002,28595236,0,0,8.374213,41.714805,0,0


In [127]:
conditions = [(df['Trend'] == 1) & (df['signal1'] == 1),
              (df['Trend'] == 2) & (df['signal1'] == 2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

In [128]:
df[(df['Trend'] == 1) & (df['signal1'] == 1)]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ATR,RSI,signal1,Trend,result
Date,,,,,,,,,,,,
2020-12-29,276.950012,280.510010,276.279999,276.779999,16382995,0,0,7.311811,52.016325,1,1,1
2021-04-30,326.143494,329.820007,324.500000,325.079987,26332423,0,0,8.092805,68.016962,1,1,1
2021-08-31,379.950012,382.759003,378.799988,379.380005,12345350,0,0,7.028918,65.954314,1,1,1
2021-12-28,346.630005,352.709991,345.200012,346.220001,16637608,0,0,10.089620,59.268428,1,1,1
2021-12-30,344.000000,347.230011,343.220001,344.359985,10593347,0,0,9.326076,57.471757,1,1,1


In [129]:
trendId=2
print(df[df['result']==trendId].result.count()/df[df['signal1']==trendId].signal1.count())
df[ (df['result']!=trendId) & (df['signal1']==trendId) ] # false positives

#histrs = df[ (df['result']==2) & (df['signal1']==2) ].RSI # false positives
#import matplotlib.pyplot as plt
#plt.hist(histrs,bins=2)  # density=False would make counts
#plt.ylabel('Probability')
#plt.xlabel('RSI');

0.3333333333333333


,Open,High,Low,Close,Volume,Dividends,Stock Splits,ATR,RSI,signal1,Trend,result
Date,,,,,,,,,,,,
2020-11-10,273.100006,274.000000,264.000000,272.429993,29067691,0,0,12.219143,49.262966,2,0,0
2020-11-13,277.720001,277.760010,273.100006,276.950012,10400101,0,0,10.336126,51.523451,2,0,0
2020-11-30,276.029999,277.700012,271.010010,276.970001,16693336,0,0,7.562292,52.115919,2,0,0
2020-12-15,274.829987,276.079987,267.470001,275.549988,23979461,0,0,8.008514,48.351579,2,1,0
2021-01-08,268.309998,268.950012,263.179993,267.570007,18528251,0,0,7.425729,44.519981,2,1,0
2021-09-16,372.790009,373.559998,369.709991,373.059998,11934217,0,0,6.449076,53.089087,2,1,0
2022-01-10,325.309998,328.339996,315.429993,328.070007,24942383,0,0,10.529674,45.629417,2,1,0
2022-03-15,190.800003,192.279999,185.910995,192.029999,31721682,0,0,9.155707,34.465923,2,0,0
2022-05-23,195.190002,196.992401,191.240005,196.229996,25059161,0,0,9.747976,46.910997,2,0,0


In [130]:
dfpl = df.loc[:'2021-03-20']
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.show()